In [15]:
## loading essential libraries
import numpy as np, os.path, pandas as pd, sys, matplotlib.pyplot as plt, copy, time
from pytorch import run_final_model, run_cross_valid, combined_plot, save_outputs, stat_new_data
from pytorch import feature_slist, feature_list, stat, specific, explore, print_optimal_tables, print_stat_tables, combined_plot_noise, individual_plot_noise, learning_curve, plot_learning_curve, load_model
from read_in_out import initiate_data, initiate_RNN_data, analysis_options, ML_analysis, compare_braced_input_data, compare_braced_output_data, introduce_marker_drop
from joblib import Parallel, delayed
from pytorch_utilities import transformer
feat_order = ['JA','JM','JRF']
feat_order_long = ['Joint angles', 'Joint moments', 'Joint reaction forces']

import seaborn as sns

## 9 april
## ensure to activate conda gamma

In [16]:
### some scripts need to compute
def initiate_naive_braced(dfo, dfn):
    dfo.naive_braced         = copy.deepcopy(dfn.naive)  
    dfo.naive_braced.arg     = copy.deepcopy(dfo.naive.arg)  ## creating an instance of naive_braced
    dfo.naive_braced.arch    = copy.deepcopy(dfo.naive.arch)  ## creating an instance of naive_braced
    dfo.naive_braced.save_name = copy.deepcopy(dfo.naive.save_name)  ## creating an instance of naive_braced
    dfo.naive_braced.subject = 'naive_braced'
    return dfo

def pull_other_hyp(df, old_index,  new_epoch): 
    tt = df.hyper.loc[old_index].to_dict()
    print(f'old epoch: {tt['epoch']}')
    tt['epoch'] = new_epoch    
    tmp = df.hyper[(df.hyper[list(tt)] == pd.Series(tt)).all(axis=1)]
    print(tmp.index[0])
    return tmp.index[0]

def train_final_models(D):
    ## train final model with best-avg-validation accuracy
    for i in range(3):
        specific(D.exposed,i)
        specific(D.naive  ,i)
    return None

def compute_stat(f):
    for D in f:
        for i in range(3):
            D.exposed = stat(D.exposed,i)
            print(D.exposed.subject, 10*'--***--',i)
            D.naive   = stat(D.naive,i)
            print(D.naive.subject, 10*'--***--',i)
            try:
                D.exposed_unseen.subject = 'exposed_unseen'
                D.exposed_unseen = stat(D.exposed_unseen, i)
                print('exposed_unseen', 10*'--***--',i)
            except:
                None
            try:
                D.naive_braced = stat(D.naive_braced, i)
                print('naive_braced', 10*'--***--',i)
            except:
                None
            print(f"\n\n")
    return f


def overall_stat(df):
    dff = pd.concat(df.pc,axis=1).values.flatten()
    print('pc',np.around(np.mean(dff),2), np.around(np.std(dff),2 ))
    dff = pd.concat(df.NRMSE,axis=1).values.flatten()
    print('NRMSE',np.around(np.mean(dff),2), np.around(np.std(dff),2 ))
    return None


from itertools import chain
def merge_lists(nested_list):
    return list(chain.from_iterable(nested_list))



def plot_final_results(df1, save_name, plot_what, remove_legend=False):
    print(f'plotting {save_name}, {plot_what} ........ below ....\n')
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']*10  # blue, orange, green
    labels = df1[0].feature_l
    print(labels)
    if plot_what == 'pc':
        fig, ax = plt.subplots(figsize=(9,5))
        df = [t.pc for t in df1]
        ax.set_ylabel('pearson correlation', size=12)    
        data = [data_dict[key].values.ravel().tolist() for data_dict in df for key in data_dict.keys()]
        box_width = 0.5
        pos = [i for sl in [np.arange(3)+4.5*i for i in range(len(data)//3)] for i in sl]
        bplot = ax.boxplot(data, positions=pos, patch_artist=True, whis=None, showfliers=False,medianprops=dict(color='black'))    
        for box, color in zip(bplot['boxes'], colors):
            box.set_facecolor(color)
        ax.set_ylim(-1.02,1.02)
        for patch, color in zip(bplot['boxes'], colors):
            patch.set_facecolor(color)
        xtlabels = [t.save_name for t in df1]
        ax.set_xticks(pos[1::3])
        ax.set_xticklabels(xtlabels,rotation=0)
        ax.tick_params(axis='both', labelsize=11.5)  
        ax.grid(axis='y', color='lightgray', linestyle='-', alpha=0.3)
        from matplotlib.patches import Patch
        legend_patches = [Patch(facecolor=c, label=l) for c, l in zip(colors, labels)]
        legend = ax.legend(handles=legend_patches,ncol=3,loc=(0.2, 1.01),  columnspacing=1.5)  # You can change the location as needed
        legend.get_frame().set_alpha(1)    
        if remove_legend:
            legend.remove()
        plt.tight_layout()
        plt.savefig(f'./plots_out/{save_name}.{plot_what}.pdf', dpi=600)
        plt.show()
        plt.close()

    elif plot_what == 'nparams':
        fig, ax = plt.subplots(figsize=(9,4))
        data = merge_lists([t.nparams for t in df1])
        ax.set_ylabel('# parameters', size=12)    
        pos = [i for sl in [np.arange(3)+4.5*i for i in range(len(data)//3)] for i in sl]
        ax.bar(pos, data, color = colors*20)
        ax.set_xlim(pos[0]-0.5, pos[-1]+0.5)
        xtlabels = [t.save_name for t in df1]
        ax.set_xticks(pos[1::3])
        ax.set_xticklabels(xtlabels,rotation=0)
        ax.tick_params(axis='both', labelsize=11.5)  
        ax.grid(axis='y', color='lightgray', linestyle='-', alpha=0.3)
        from matplotlib.patches import Patch
        legend_patches = [Patch(facecolor=c, label=l) for c, l in zip(colors, labels)]
        legend = ax.legend(handles=legend_patches,ncol=3,loc=(0.2, 1.01),  columnspacing=1.5)  # You can change the location as needed
        legend.get_frame().set_alpha(1) 
        if remove_legend:
            legend.remove()
        plt.tight_layout()
        plt.savefig(f'./plots_out/{save_name}.{plot_what}.pdf', dpi=600)
        plt.show()
        plt.close()

    elif plot_what in ['RMSE', 'NRMSE']:
        if plot_what == 'NRMSE':
            df = [t.NRMSE  for t in df1[1:]]
        elif plot_what == 'RMSE':
            df = [t.RMSE for t in df1[1:]]
        data = [data_dict[key].values.ravel().tolist() for data_dict in df for key in data_dict.keys()]
        box_width = 0.5
        pos = [i for sl in [np.arange(3)+4.5*i for i in range(len(data)//3)] for i in sl]
        xtlabels = [t.save_name for t in df1[1:]]
        for i in np.arange(3):
            fig, ax = plt.subplots(figsize=(9,5))
            ax.set_ylabel(plot_what, size=12)    
            posi    = pos[i::3]
            datai   = data[i::3]
            colorsi = colors[i::3]
            bplot  = ax.boxplot(datai, positions=posi, patch_artist=True, whis=None, showfliers=False,medianprops=dict(color='black'))    
            for box, color in zip(bplot['boxes'], colorsi):
                box.set_facecolor(color)
            for patch, color in zip(bplot['boxes'], colorsi):
                patch.set_facecolor(color)
            # ax.set_ylim(-.2,3.02)
            ax.set_xticks(posi)
            ax.set_xticklabels(xtlabels,rotation=0)
            ax.tick_params(axis='both', labelsize=11.5)  
            ax.grid(axis='y', color='lightgray', linestyle='-', alpha=0.3)
            from matplotlib.patches import Patch
            legend_patches = [Patch(facecolor=c, label=l) for c, l in zip(colorsi, labels[i:i+1])]
            legend = ax.legend(handles=legend_patches,ncol=1,loc=(0.4, 1.01), columnspacing=1.5)  # You can change the location as needed
            legend.get_frame().set_alpha(1) 
            if remove_legend:
                legend.remove()
            plt.tight_layout()
            plt.savefig(f'./plots_out/{save_name}.{plot_what}.{i}.pdf', dpi=600)
            plt.show()
            plt.close()



In [17]:
### function to load_model_index
def load_model_index(fm, fm_braced, data_kind):
    if 'LM' in data_kind:
        fm.LM.exposed.arg      = [43, 43, 43]
        fm.LM.naive.arg        = [43, 43, 43]
        fm.LM.exposed.arch, fm.LM.naive.arch  = ['LM']*3, ['LM']*3
        fm.LM.exposed_unseen     = copy.deepcopy(fm.LM.exposed)
        fm.LM.save_name, fm.LM.exposed.save_name, fm.LM.naive.save_name, fm.LM.exposed_unseen.save_name = ('LM',)*4
        fm.LM = initiate_naive_braced(fm.LM, fm_braced.LM) ## creating an instance of naive_braced
        
    if 'rf' in data_kind:
        fm.rf.exposed.arg      = [6, 12, 6]
        fm.rf.naive.arg        = [0, 12, 20]
        fm.rf.exposed.arch, fm.rf.naive.arch = ['rf']*3, ['rf']*3
        fm.rf.exposed_unseen     = copy.deepcopy(fm.rf.exposed)
        fm.rf.save_name, fm.rf.exposed.save_name, fm.rf.naive.save_name, fm.rf.exposed_unseen.save_name = ('RF',)*4
        fm.rf = initiate_naive_braced(fm.rf, fm_braced.rf) ## creating an instance of naive_braced

    if 'transformer' in data_kind:
        fm.transformer.exposed.arg      = [23,23,23]
        fm.transformer.naive.arg        = [23,23,23]
        fm.transformer.exposed.arch, fm.transformer.naive.arch = ['transformer']*3, ['transformer']*3
        fm.transformer.exposed_unseen     = copy.deepcopy(fm.transformer.exposed)
        fm.transformer.save_name, fm.transformer.exposed.save_name, fm.transformer.naive.save_name, fm.transformer.exposed_unseen.save_name = ('transformer',)*4
        fm.transformer = initiate_naive_braced(fm.transformer, fm_braced.transformer) ## creating an instance of naive_braced
            
    if 'xgbr' in data_kind:
        # fm.xgbr.exposed.arg        = [218, 218, 218]  ## prefinal
        # fm.xgbr.naive.arg          = [218, 218, 218]
        fm.xgbr.exposed.arg        = [96, 96, 96]
        fm.xgbr.naive.arg          = [96, 96, 218]
        fm.xgbr.exposed.arg        = [96, 96, 96]
        fm.xgbr.naive.arg          = [96, 46, 218]
        fm.xgbr.exposed.arch, fm.xgbr.naive.arch = ['xgbr']*3, ['xgbr']*3
        fm.xgbr.exposed_unseen     = copy.deepcopy(fm.xgbr.exposed)
        fm.xgbr.save_name, fm.xgbr.exposed.save_name, fm.xgbr.naive.save_name, fm.xgbr.exposed_unseen.save_name = ('XGBR',)*4
        fm.xgbr = initiate_naive_braced(fm.xgbr, fm_braced.xgbr) ## creating an instance of naive_braced

    if 'NN' in data_kind:
        fm.NN.exposed.arg        = [1043, 1043, 81]
        fm.NN.naive.arg          = [1043, 1091, 1046] ## 1043, 1091, 1043
        fm.NN.exposed.arch, fm.NN.naive.arch = ['NN']*3, ['NN']*3
        fm.NN.exposed_unseen     = copy.deepcopy(fm.NN.exposed)
        fm.NN.save_name, fm.NN.exposed.save_name, fm.NN.naive.save_name, fm.NN.exposed_unseen.save_name = ('FFNN',)*4
        fm.NN = initiate_naive_braced(fm.NN, fm_braced.NN) ## creating an instance of naive_braced

    if 'CNNLSTM' in data_kind:
        fm.CNNLSTM.exposed.arg        = [387, 387, 23]
        fm.CNNLSTM.naive.arg          = [387, 387, 391] ## 389
        fm.CNNLSTM.exposed.arch, fm.CNNLSTM.naive.arch = ['CNNLSTM']*3, ['CNNLSTM']*3
        fm.CNNLSTM.exposed_unseen     = copy.deepcopy(fm.CNNLSTM.exposed)
        fm.CNNLSTM.save_name, fm.CNNLSTM.exposed.save_name, fm.CNNLSTM.naive.save_name, fm.CNNLSTM.exposed_unseen.save_name = ('CNNLSTM',)*4
        fm.CNNLSTM = initiate_naive_braced(fm.CNNLSTM, fm_braced.CNNLSTM) ## creating an instance of naive_braced
    
    if 'RNN' in data_kind:    
        fm.VRNN = copy.deepcopy(fm.RNN) 
        fm.LSTM = copy.deepcopy(fm.RNN) 
        fm.GRU  = copy.deepcopy(fm.RNN) 
        
        fm.VRNN.exposed.arg       = [3119, 3411, 3125] ## 3123
        fm.VRNN.naive.arg         = [3169, 2051, 3411]  #100
        fm.VRNN.exposed.arch, fm.VRNN.naive.arch = ['RNN']*3, ['RNN']*3   ## (SimpleRNN, LSTM, GRU)
        fm.VRNN.exposed_unseen     = copy.deepcopy(fm.VRNN.exposed)
        fm.VRNN.save_name, fm.VRNN.exposed.save_name, fm.VRNN.naive.save_name, fm.VRNN.exposed_unseen.save_name = ('VRNN',)*4
        fm.VRNN = initiate_naive_braced(fm.VRNN, fm_braced.RNN) ## creating an instance of naive_braced
    
        fm.LSTM.exposed.arg       = [3458,3669,4396] ## 3458 -- make cnnlstm here0 ..
        fm.LSTM.naive.arg         = [3458,3669,4396] #[7493,4108, 4396 ]
        fm.LSTM.exposed.arch, fm.LSTM.naive.arch = ['RNN']*3, ['RNN']*3 
        fm.LSTM.exposed_unseen     = copy.deepcopy(fm.LSTM.exposed)
        fm.LSTM.save_name, fm.LSTM.exposed.save_name, fm.LSTM.naive.save_name, fm.LSTM.exposed_unseen.save_name = ('LSTM',)*4
        fm.LSTM = initiate_naive_braced(fm.LSTM, fm_braced.RNN) ## creating an instance of naive_braced
    
        fm.GRU.exposed.arg        = [9153, 10029, 10301]
        fm.GRU.naive.arg          = [9845, 8309, 6916 ]
        fm.GRU.exposed.arch, fm.GRU.naive.arch = ['RNN']*3, ['RNN']*3   ## (SimpleRNN, LSTM, GRU)
        fm.GRU.exposed_unseen     = copy.deepcopy(fm.GRU.exposed)
        fm.GRU.save_name, fm.GRU.exposed.save_name, fm.GRU.naive.save_name, fm.GRU.exposed_unseen.save_name = ('GRU',)*4
        fm.GRU = initiate_naive_braced(fm.GRU, fm_braced.RNN) ## creating an instance of naive_braced

    if 'CNN' in data_kind:
        fm.CNN.exposed.arg        = [28, 243, 31]  
        fm.CNN.naive.arg          = [36,  211, 217] 
        fm.CNN.exposed.arch, fm.CNN.naive.arch = ['CNN']*3, ['CNN']*3
        fm.CNN.exposed_unseen     = copy.deepcopy(fm.CNN.exposed)
        fm.CNN.save_name, fm.CNN.exposed.save_name, fm.CNN.naive.save_name, fm.CNN.exposed_unseen.save_name = ('CNN',)*4
        fm.CNN = initiate_naive_braced(fm.CNN, fm_braced.CNN) ## creating an instance of naive_braced
    print(f'Here argument n is n+2th line in the file ... ')

    if 'ResNet' in data_kind:
        fm.ResNet.exposed.arg        = [218, 243, 406]  
        fm.ResNet.naive.arg          = [7,  13, 56]
        fm.ResNet.exposed.arch, fm.ResNet.naive.arch = ['ResNet']*3, ['ResNet']*3
        fm.ResNet.exposed_unseen     = copy.deepcopy(fm.ResNet.exposed)
        fm.ResNet.save_name, fm.ResNet.exposed.save_name, fm.ResNet.naive.save_name, fm.ResNet.exposed_unseen.save_name = ('ResNet',)*4
        fm.ResNet = initiate_naive_braced(fm.ResNet, fm_braced.ResNet) ## creating an instance of naive_braced

    if 'convLSTM' in data_kind:
        fm.convLSTM.exposed.arg        = [218, 243, 406]  
        fm.convLSTM.naive.arg          = [7,  13, 56]
        fm.convLSTM.exposed.arch, fm.convLSTM.naive.arch = ['convLSTM']*3, ['convLSTM']*3
        fm.convLSTM.exposed_unseen     = copy.deepcopy(fm.convLSTM.exposed)
        fm.convLSTM.save_name, fm.convLSTM.exposed.save_name, fm.convLSTM.naive.save_name, fm.convLSTM.exposed_unseen.save_name = ('convLSTM',)*4
        fm.convLSTM = initiate_naive_braced(fm.convLSTM, fm_braced.convLSTM) ## creating an instance of naive_braced

    return fm
time_taken_model_wise = {}


In [13]:
#####################################################################    
#######        FINAL BLOCK .. only use for plotting acccuracy results #########
#####################################################################            
plot_acccuracy_results =0 
if plot_acccuracy_results:
    
    window=1
    mm_list1 =  ['LM', 'NN', 'rf', 'xgbr']
    fm1 = ML_analysis('final_model_list', mm_list1, window)
    fmb1 = ML_analysis('final_model_list', mm_list1, window, add1 = 'Braced_')
    fm1 = load_model_index(fm1, fmb1, mm_list1)
    
    
    window=5
    mm_list2 = ['CNN', 'CNNLSTM', 'convLSTM', 'RNN', 'RNN', 'RNN' ]
    fm2 = ML_analysis('final_model_list', mm_list2, window)
    fmb2 = ML_analysis('final_model_list', mm_list2, window, add1 = 'Braced_')
    fm2 = load_model_index(fm2, fmb2, mm_list2)
    
    model1 = [fm1.LM, fm1.NN, fm1.rf, fm1.xgbr]  
    model2 = [fm2.CNN, fm2.CNNLSTM, fm2.convLSTM, fm2.VRNN, fm2.LSTM, fm2.GRU]
    fm1 = compute_stat(model1)
    fm2 = compute_stat(model2)
    
    
    
    for m in model1+model2:
        print_stat_tables(m)
    for plot_what in ['nparams', 'pc','NRMSE','RMSE']:
        plot_final_results([m.exposed for m in model1+model2], 'final.exposed', plot_what)
        plot_final_results([m.exposed_unseen for m in model1+model2], 'final.exposed_unseen', plot_what)
        plot_final_results([m.naive for m in model1+model2], 'final.naive', plot_what)
        if plot_what in ['pc','NRMSE','RMSE']:
            plot_final_results([m.naive_braced for m in model1+model2], 'final.naiveb', plot_what)


In [12]:
##### individal noise plot final ..16 march 2025
#####################################################################    
#######        FINAL BLOCK .. only use for individual noise results ###
#####################################################################


individual_noise_results = 0
if individual_noise_results:

    window=5
    mm_list2 = ['convLSTM']
    fm2 = ML_analysis('final_model_list', mm_list2, window)
    fmb2 = ML_analysis('final_model_list', mm_list2, window, add1 = 'Braced_')
    fm2 = load_model_index(fm2, fmb2, mm_list2)
    individual_plot_noise(fm2.convLSTM, 0)
    individual_plot_noise(fm2.convLSTM, 1)
    individual_plot_noise(fm2.convLSTM, 2)

#####################################################################    
#######        Return overall stat 
#####################################################################

    window=5
    mm_list2 = ['convLSTM']
    fm2 = ML_analysis('final_model_list', mm_list2, window)
    fmb2 = ML_analysis('final_model_list', mm_list2, window, add1 = 'Braced_')
    fm2 = load_model_index(fm2, fmb2, mm_list2)
    fm2 = compute_stat([fm2.convLSTM])
    
    uu = fm2[0]
    overall_stat(uu.exposed)
    overall_stat(uu.exposed_unseen)
    overall_stat(uu.naive)
    overall_stat(uu.naive_braced)
    




In [11]:
# (351, 5, 58)
### completed on 10 March
#####################################################################    
#######        FINAL BLOCK .. only use for complete noise results ###
#####################################################################            
Added_noise_results=0
if Added_noise_results: 

    window=5
    mm = 'CNNLSTM'
    fm = ML_analysis('final_model_list', [f'{mm}'], window)
    fmb = ML_analysis('final_model_list', [f'{mm}'], window, add1 = 'Braced_')
    fm = load_model_index(fm, fmb, [f'{mm}'])
    D = fm.CNNLSTM
    combined_plot_noise(D)
    
    mm = 'convLSTM'
    fm = ML_analysis('final_model_list', [f'{mm}'], window)
    fmb = ML_analysis('final_model_list', [f'{mm}'], window, add1 = 'Braced_')
    fm = load_model_index(fm, fmb, [f'{mm}'])
    D = fm.convLSTM
    combined_plot_noise(D)
    
    mm = 'CNN'
    fm = ML_analysis('final_model_list', [f'{mm}'], window)
    fmb = ML_analysis('final_model_list', [f'{mm}'], window, add1 = 'Braced_')
    fm = load_model_index(fm, fmb, [f'{mm}'])
    D = fm.CNN
    combined_plot_noise(D)
    
    mm = 'RNN'
    fm = ML_analysis('final_model_list', [f'{mm}'], window)
    fmb = ML_analysis('final_model_list', [f'{mm}'], window, add1 = 'Braced_')
    fm = load_model_index(fm, fmb, [f'{mm}'])
    D = fm.VRNN
    combined_plot_noise(D)
    
    mm = 'RNN'
    fm = ML_analysis('final_model_list', [f'{mm}'], window)
    fmb = ML_analysis('final_model_list', [f'{mm}'], window, add1 = 'Braced_')
    fm = load_model_index(fm, fmb, [f'{mm}'])
    D = fm.LSTM
    combined_plot_noise(D)
    
    mm = 'RNN'
    fm = ML_analysis('final_model_list', [f'{mm}'], window)
    fmb = ML_analysis('final_model_list', [f'{mm}'], window, add1 = 'Braced_')
    fm = load_model_index(fm, fmb, [f'{mm}'])
    D = fm.GRU
    combined_plot_noise(D)
    
    
    window=1
    mm1 = 'NN'
    fm1 = ML_analysis('final_model_list', [f'{mm1}'], window)
    fmb1 = ML_analysis('final_model_list', [f'{mm1}'], window, add1 = 'Braced_')
    fm1 = load_model_index(fm1, fmb1, [f'{mm1}'])
    D = fm1.NN
    combined_plot_noise(D)
    
    mm1 = 'LM'
    fm1 = ML_analysis('final_model_list', [f'{mm1}'], window)
    fmb1 = ML_analysis('final_model_list', [f'{mm1}'], window, add1 = 'Braced_')
    fm1 = load_model_index(fm1, fmb1, [f'{mm1}'])
    D = fm1.LM
    combined_plot_noise(D)
    
    mm1 = 'rf'
    fm1 = ML_analysis('final_model_list', [f'{mm1}'], window)
    fmb1 = ML_analysis('final_model_list', [f'{mm1}'], window, add1 = 'Braced_')
    fm1 = load_model_index(fm1, fmb1, [f'{mm1}'])
    D = fm1.rf
    combined_plot_noise(D)
    
    mm1 = 'xgbr'
    fm1 = ML_analysis('final_model_list', [f'{mm1}'], window)
    fmb1 = ML_analysis('final_model_list', [f'{mm1}'], window, add1 = 'Braced_')
    fm1 = load_model_index(fm1, fmb1, [f'{mm1}'])
    D = fm1.xgbr
    combined_plot_noise(D)


In [10]:
#####################################################################    
#######        FINAL BLOCK .. only use for complete noise results ###
#####################################################################            

def plot_carbon_footprint(remove_legend=False, per_epoch = False, time_taken=False):
    epochs = {'LM': [100, 100, 100, 100, 100, 100],
     'FFNN': [100, 100, 50, 100, 100, 100],
     'RF': [50, 50, 50, 50, 50, 50],
     'XGBR': [200, 200, 200, 200, 100, 500],
     'CNN': [50, 100, 50, 50, 100, 100],
     'CNNLSTM': [100, 100, 50, 100, 100, 100],
     'convLSTM': [100, 100, 200, 100, 100, 50],
     'VRNN': [100, 200, 100, 200, 100, 200],
     'LSTM': [50, 50, 50, 50, 50, 50],
     'GRU': [100, 100, 200, 100, 100, 50]}
    epochs = pd.DataFrame(epochs).T
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c']*10  # blue, orange, green
    labels = ['Joint angles', 'Joint moments', 'Joint reaction forces']
    df = pd.read_csv('training_time.best-fit.csv',index_col=0)
    epochs.columns = df.columns
    nsamples = {'naive': 15610, 'exposed':13653}
    if per_epoch:
        df = df/epochs
    df = 1.02*df/3600  ### equivalent CO2
    if time_taken:
        df = pd.read_csv('prediction_time.csv',index_col=0)
    df_exposed = []
    for ind, row in df[['exposed_0', 'exposed_1','exposed_2']].iterrows():
        df_exposed.append(list(row))
    df_exposed  = list(chain.from_iterable(df_exposed))
    df_naive = []
    for ind, row in df[['naive_0', 'naive_1','naive_2']].iterrows():
        df_naive.append(list(row))
    df_naive  = list(chain.from_iterable(df_naive))
    for data, save_name in zip([df_naive, df_exposed], ['naive', 'exposed']):
        fig, ax = plt.subplots(figsize=(9,4))
        if per_epoch:
            ax.set_ylabel(r'Grams of CO$_2$ equivalent (per epoch)',size=12)
        elif time_taken:
            ax.set_ylabel(r'prediction time per frame (in ms)',size=12)
        else:
            ax.set_ylabel(r'Grams of CO$_2$ equivalent',size=12)            
        pos = [i for sl in [np.arange(3)+4.5*i for i in range(len(data)//3)] for i in sl]
        ax.bar(pos, data, color = colors*20)
        ax.set_xlim(pos[0]-0.5, pos[-1]+0.5)
        xtlabels = df.index
        ax.set_xticks(pos[1::3])
        ax.set_xticklabels(xtlabels,rotation=0)
        ax.tick_params(axis='both', labelsize=11.5)
        ax.grid(axis='y', color='lightgray', linestyle='-', alpha=0.3)
        from matplotlib.patches import Patch
        legend_patches = [Patch(facecolor=c, label=l) for c, l in zip(colors, labels)]
        legend = ax.legend(handles=legend_patches,ncol=3,loc=(0.2, 1.01),  columnspacing=1.5)  # You can change the location as needed
        legend.get_frame().set_alpha(1)
        if remove_legend:
            legend.remove()
        plt.tight_layout()
        if per_epoch:
            plt.savefig(f'./plots_out/Carbon_footprint.{save_name}.per_epoch.pdf', dpi=600)
        elif time_taken:
            plt.savefig(f'./plots_out/Carbon_footprint.{save_name}.time_taken.pdf', dpi=600)
        else:
            plt.savefig(f'./plots_out/Carbon_footprint.{save_name}.pdf', dpi=600)
        plt.show()
        plt.close()

# plot_carbon_footprint(remove_legend=0, per_epoch=True)
# plot_carbon_footprint(remove_legend=1, per_epoch=False)
# plot_carbon_footprint(remove_legend=1, time_taken=1)



In [9]:
# # ####################################################################    
# # ######        FINAL BLOCK .. only use for SHAP ###
# # ####################################################################            
model_explainability_analysis = False
if model_explainability_analysis:
    
    import shap
    
    markers_list = ['C7','T10','CLAV','STRN','RBAK','LSHO','RSHO','RUPA','RELB','RFRM','RWRA','RWRB','RFIN','RASI','LPSI','RPSI','D1','D2','D3']
    markers_list_coords = [m+k for m in markers_list for k in ['_x','_y','_z']] + ['time']
    col_list = [str(i) for i in np.arange(58)]
    fm = ML_analysis('final_model_list', ['xgbr'], 1)
    fmb = ML_analysis('final_model_list', ['xgbr'], 1, add1 = 'Braced_')
    fm = load_model_index(fm, fmb, ['xgbr'])
    X1 = fm.xgbr.data.subject_exposed('JA',0)
    X = X1.train_in
    X.columns = markers_list_coords
    
    
    
    model = load_model('exposed','JA','rf','6')
    explainer = shap.TreeExplainer(model)
    shap_values1 = explainer.shap_values(X)
    print('JA loaded....')
    
    model = load_model('exposed','JM','rf','12')
    explainer = shap.TreeExplainer(model)
    shap_values2 = explainer.shap_values(X)
    print('JM loaded....')
    
    model = load_model('exposed','JRF','rf','6')
    explainer = shap.TreeExplainer(model)
    shap_values3 = explainer.shap_values(X)
    print('JRF loaded....')
    
    title = {0: ['SFE',	'SAA',	'SIR',	'EFE',	'EPS',	'WFE'	,'WAA',	'TFE',	'TAA',	'TIR'],
             1: ['SacrumPelvisFlexionExtensionMoment'	,'SacrumPelvisAxialMoment'	,'SacrumPelvisLateralMoment',	'GlenoHumeralFlexion'	,'GlenoHumeralAbduction',	
                     'GlenoHumeralExternalRotation'	,'ElbowFlexion',	'ElbowPronation',	'WristFlexion',	'WristAbduction'],
             2: ['TML'	,'TPD'	,'TAP',	'GML',	'GPD',	'GAP',	'EML',	'EPD'	,'EAP',	'WML',	'WPD',	'WAP']}
    
    ff = ['Joint angles','Joint moments','Joint reaction forces']
    ffs = ['JA','JM','JRF']
    size = 19
    
    for enum, shpv in enumerate([shap_values1, shap_values2, shap_values3]):
        for i in np.arange(shpv.shape[2]):
            plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
            ax=shap.summary_plot(shpv[:,:,i], X, plot_type="bar",max_display=16, show=False)
            plt.title(f'{ff[enum]} ({title[enum][i]})',size=size)
            plt.xticks(fontsize=size)  # Set font size of x-axis tick labels
            plt.yticks(fontsize=size)  # Set font size of x-axis tick labels
            xlabel = plt.xlabel('mean(|SHAP value|) (avg. impact on prediction magnitude)',size=size,labelpad=8)
            ylabel = plt.ylabel('feature importance',size=size,labelpad=8)
            xlabel.set_position((0.38, xlabel.get_position()[1]))
            plt.tight_layout()
            plt.savefig(f'./plots_out/rf.{ffs[enum]}.{i+1}.pdf', dpi=600, bbox_inches='tight')
            plt.show()
            plt.close()
    


In [8]:
### following generates the data for learning curve

from pytorch import *
from pytorch_utilities import *
from pytorch import run_NN, evaluate_mse
import random 


def learning_curve(fm):
    ## learning curve are done using all the data i.e. validation accuracy is essentially test accuracy
    res = analysis_options("results for learning curve -- note only for naive models")
    res.model = fm.what
    res.save_name = fm.save_name 
    res.subject = 'naive'
    nval = np.arange(10)  ### this allows picking random subjects to initialze or repeat the computation multiple times (with same subejcts) to check robustness
    res.RMSE_train = {}
    res.RMSE_test  = {}
    for enumf, feat in enumerate(['JRF','JM','JA']):
    # for enumf, feat in enumerate(fm.feature):
        print(f'Ongoing feature .... {feat}')
        hyper_arg = fm.naive.arg[enumf]
        model_class = fm.naive.arch[enumf]
        hyper_val = fm.hyper.loc[hyper_arg]
        norm_out  = hyper_val['norm_out']
        data = fm.data.subject_naive(feat,norm_out)
        nsub = len(data.train_in_list)
        # nsub=1        
        res.RMSE_train[feat] = pd.DataFrame(index = np.arange(1, nsub+1), columns=nval)  ### nsub -- rows with n subjects in the training 
        res.RMSE_test[feat]  = pd.DataFrame(index = np.arange(1, nsub+1), columns=nval)  ### nval -- cols with n indep trials in the training 
        for r in nval:
            print(f'Ongoing trials .... {fm.feature}')
            for n in np.arange(1,nsub+1):

                if n ==1:    
                    for lt in [100, 300, 500]:
                        rand = random.sample(range(0, nsub), k=n)
                        tmp_train_in  = [ data.train_in_list[f][0:lt] for f in rand]
                        tmp_train_out = [ data.train_out_list[f][0:lt] for f in rand]
                        try:
                            X = pd.concat(tmp_train_in)
                        except:
                            X = np.concatenate(tmp_train_in)  
                        Y = pd.concat(tmp_train_out)    
                        print(X.shape)
                        print(Y.shape)
                        model = run_NN(X, Y, data.test_in, data.test_out, hyper_val,  model_class, verbose = 1)
                        try:
                            Y_trainp = model.predict(X,verbose=2)
                            res.RMSE_train[feat].loc[lt+n,r] = root_mean_squared_error(Y_trainp,Y).numpy()   ### test loss = 0 and test accuracy 1
                            Y_testp = model.predict(data.test_in,verbose=2)
                            res.RMSE_test[feat].loc[lt+n,r] = root_mean_squared_error(Y_testp,data.test_out).numpy()   ### test loss = 0 and test accuracy 1
                        except:
                            res.RMSE_train[feat].loc[lt+n,r] = evaluate_mse(X, Y, model)  ### test loss = 0 and test accuracy 1
                            res.RMSE_test[feat].loc[lt+n,r]  = evaluate_mse(data.test_in, data.test_out, model)  ### test loss = 0 and test accuracy 1
        
                        print('subjects_picked:',rand)
                        print(feat, lt+n, res.RMSE_train[feat].loc[lt+n,r], res.RMSE_test[feat].loc[lt+n,r])
                        print('****'*8)
                        del model
    
                rand = random.sample(range(0, nsub), k=n)
                tmp_train_in  = [ data.train_in_list[f]  for f in rand]
                tmp_train_out = [ data.train_out_list[f] for f in rand]
                try:
                    X = pd.concat(tmp_train_in)
                except:
                    X = np.concatenate(tmp_train_in)  
                Y = pd.concat(tmp_train_out)    
                print(X.shape)
                print(Y.shape)
                model = run_NN(X, Y, data.test_in, data.test_out, hyper_val,  model_class, 1)
                
                if res.model in ['convLSTM', 'LM']:
                    Y_trainp = model.predict(X,verbose=2)
                    res.RMSE_train[feat].loc[n,r] = root_mean_squared_error(Y_trainp,Y).numpy()   ### test loss = 0 and test accuracy 1
                    Y_testp = model.predict(data.test_in,verbose=2)
                    res.RMSE_test[feat].loc[n,r] = root_mean_squared_error(Y_testp,data.test_out).numpy()   ### test loss = 0 and test accuracy 1
                elif res.model in ['rf']:
                    res.RMSE_train[feat].loc[n,r] = np.sqrt(evaluate_mse(X, Y, model))  ### test loss = 0 and test accuracy 1
                    res.RMSE_test[feat].loc[n,r]  = np.sqrt(evaluate_mse(data.test_in, data.test_out, model))  ### test loss = 0 and test accuracy 1
                del model
                print('subjects_picked:',rand)
                print(feat, n, res.RMSE_train[feat].loc[n,r], res.RMSE_test[feat].loc[n,r])
                print('****'*8)

        
        res.RMSE_train[feat].to_csv(f'./lc_data/lc.{res.model}.{res.subject}.{feat}.{res.save_name}.train.txt')
        res.RMSE_test[feat].to_csv( f'./lc_data/lc.{res.model}.{res.subject}.{feat}.{res.save_name}.test.txt' )
        ## columns are various nval trials and rows are number of subjects
    return res, fm


# window=5
# mm = 'convLSTM'
# fm = ML_analysis('final_model_list', [f'{mm}'], window)
# fmb = ML_analysis('final_model_list', [f'{mm}'], window, add1 = 'Braced_')
# fm = load_model_index(fm, fmb, [f'{mm}'])
# D = fm.convLSTM
# a,b = learning_curve(D)

# window=1
# mm = 'rf'
# fm = ML_analysis('final_model_list', [f'{mm}'], window)
# fmb = ML_analysis('final_model_list', [f'{mm}'], window, add1 = 'Braced_')
# fm = load_model_index(fm, fmb, [f'{mm}'])
# D = fm.rf
# a,b = learning_curve(D)

# window=1
# mm = 'LM'
# fm = ML_analysis('final_model_list', [f'{mm}'], window)
# fmb = ML_analysis('final_model_list', [f'{mm}'], window, add1 = 'Braced_')
# fm = load_model_index(fm, fmb, [f'{mm}'])
# D = fm.LM
# a,b = learning_curve(D)


# window=1
# mm = 'NN'
# fm = ML_analysis('final_model_list', [f'{mm}'], window)
# fmb = ML_analysis('final_model_list', [f'{mm}'], window, add1 = 'Braced_')
# fm = load_model_index(fm, fmb, [f'{mm}'])
# D = fm.NN
# a,b = learning_curve(D)

# window=5
# mm = 'CNN'
# fm = ML_analysis('final_model_list', [f'{mm}'], window)
# fmb = ML_analysis('final_model_list', [f'{mm}'], window, add1 = 'Braced_')
# fm = load_model_index(fm, fmb, [f'{mm}'])
# D = fm.CNN
# a,b = learning_curve(D)


In [7]:
### following generates the plot for learning curve

def plot_learning_curve(model_kind, subject_kind, feat):
    import matplotlib.ticker as ticker

    alpha = 0.2
    color = ['r','b']
    scale = {'JA':10, 'JM':10, 'JRF':12}
    index = feature_slist.index(feat)
    yl = feature_list[index]
    s = 14
    ind_order = [101, 301, 501, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
    train_err = pd.read_csv(f'./lc_data/lc.{model_kind}.{subject_kind}.{feat}.{model_kind}.train.txt', index_col=0)
    val_err   = pd.read_csv(f'./lc_data/lc.{model_kind}.{subject_kind}.{feat}.{model_kind}.test.txt', index_col=0)    
    # print(val_err)
    train_err = train_err.loc[ind_order]/scale[feat]
    val_err   = val_err.loc[ind_order]/scale[feat]
    nsub = train_err.shape[0]
    fig, ax = plt.subplots(figsize=(7,4))
    ind = np.arange(1,nsub+1)
    label = ['Training', 'Validation']
    for enum,d in enumerate([train_err,val_err]):
        ax.scatter(ind, d.mean(axis=1), color=color[enum], label=label[enum])
        ax.plot(ind, d.mean(axis=1), color=color[enum], label='_no_legend_')
        ax.fill_between(ind, d.mean(axis=1)+d.std(axis=1), d.mean(axis=1)-d.std(axis=1), facecolor=color[enum], alpha=alpha)
    ax.legend(fontsize=s)
    ax.set_xticks(np.arange(1,1+len(ind_order)))  # Set the positions of the ticks
    ind_order = [0.1, 0.3, 0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
    ax.set_xticklabels(ind_order)  # Set the labels for the ticks
    ax.tick_params(axis='y', labelsize=s,   pad=4,length=3,width=0.5,direction= 'inout',which='major',rotation=0)
    ax.tick_params(axis='x', labelsize=s,   pad=4,length=3,width=0.5,direction= 'inout',which='major',rotation=30)
    ax.set_xlabel("# of subjects", fontsize=s)
    ax.set_ylabel(f"RMSE loss ({yl})", fontsize=s)
    ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
    plt.savefig('./plots_out/' + model_kind + '.' + subject_kind+ '.' + feat + '.lc.pdf', dpi=600)
    plt.ylim(-1,10)
    plt.show()
    plt.close()
    
### following generates the plot for learning curve
    
# plot_learning_curve('LM','naive','JA')
# plot_learning_curve('LM','naive','JM')
# plot_learning_curve('LM','naive','JRF')

# plot_learning_curve('convLSTM','naive','JA')
# plot_learning_curve('convLSTM','naive','JM')
# plot_learning_curve('convLSTM','naive','JRF')

# plot_learning_curve('rf','naive','JA')
# plot_learning_curve('rf','naive','JM')
# plot_learning_curve('rf','naive','JRF')

In [3]:

##### individal noise plot final ...
#####################################################################    
#######        FINAL BLOCK .. to generate latex hyperparameter tables#
#####################################################################

individal_noise_plot_final = False
if individal_noise_plot_final:
    
    window=1
    mm_list1 =  ['LM', 'NN', 'rf', 'xgbr']
    fm1 = ML_analysis('final_model_list', mm_list1, window)
    fmb1 = ML_analysis('final_model_list', mm_list1, window, add1 = 'Braced_')
    fm1 = load_model_index(fm1, fmb1, mm_list1)
    
    
    window=5
    mm_list2 = ['CNN', 'CNNLSTM', 'convLSTM', 'RNN', 'RNN', 'RNN' ]
    fm2 = ML_analysis('final_model_list', mm_list2, window)
    fmb2 = ML_analysis('final_model_list', mm_list2, window, add1 = 'Braced_')
    fm2 = load_model_index(fm2, fmb2, mm_list2)
    
    
    
    epochs = {}
    for D in [fm1.LM, fm1.NN, fm1.rf, fm1.xgbr, fm2.CNN, fm2.CNNLSTM, fm2.convLSTM, fm2.VRNN, fm2.LSTM,fm2.GRU]: 
        uu = D.hyper.loc[D.exposed.arg + D.naive.arg]
        uu.index = [i + ' (SE)'for i in feat_order_long] +  [i + ' (SN)'for i in feat_order_long]
        uu.columns = [i.replace('_',' ') for i in uu.columns]
        uu.replace('_', ' ', regex=True, inplace=True)
        arch  = D.exposed.save_name
        see = "\\textit{subject-exposed (SE)}"
        sne = "\\textit{subject-naive (SN)}"
        try:
            epochs[arch] = uu['epoch'].to_list()
        except:
            epochs[arch] = uu['n estimators'].to_list()
        caption  = f'Hyperparameters choices explored for {NN_name_map[arch]} ({arch}) in {see} and {sne} settings and the optimal hyperparameters found for each MSK output category.'
        latex_table = uu.to_latex(index=True, caption = caption,float_format="%.3f")
        latex_table = latex_table.replace(r'\begin{tabular}', r'\centering\scalebox{0.6}{\begin{tabular}')
        latex_table = latex_table.replace(r'\end{tabular}', r'\end{tabular}}')
        print(latex_table)
